<a href="https://colab.research.google.com/github/viniciusrpb/natural_language_processing/blob/main/lstm_imdb_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook

import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.text import *
from fastai.text.all import *

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#8) [Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_clas'),Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/models')]

In [5]:
(path/'train').ls()

path_train = path/"train"
#path_test = path/"test"

training_set = Path(path_train)

print(training_set)

/root/.fastai/data/imdb/train


### Ajuste do Modelo de Linguagem

In [6]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [7]:
dls_lm.show_batch(max_n=2)

,text,text_
0,xxbos xxmaj monk is my favorite xxup tv show . i wait for it all week until it 's on xxmaj friday pm . xxmaj is it on any other network besides xxup usa ? i will watch the same shows over and over if they 're on . i love the all day marathons . i spent the entire day xxmaj new xxmaj years xxmaj day watching the xxmaj monk xxmaj marathon ! xxmaj tony xxmaj shaloub is wonderful,xxmaj monk is my favorite xxup tv show . i wait for it all week until it 's on xxmaj friday pm . xxmaj is it on any other network besides xxup usa ? i will watch the same shows over and over if they 're on . i love the all day marathons . i spent the entire day xxmaj new xxmaj years xxmaj day watching the xxmaj monk xxmaj marathon ! xxmaj tony xxmaj shaloub is wonderful and
1,"this film is well worth watching to witness the rebirth of human emotions and passions . xxmaj it will leave a smile on your face for days . xxbos xxmaj this is one of the movies selected to the xxup fifty xxup worst xxup films book by xxmaj harry xxmaj medved . xxmaj and while i agree it is really bad , xxmaj i 'm not sure it reached the level of wretchedness needed to merit inclusion in the book","film is well worth watching to witness the rebirth of human emotions and passions . xxmaj it will leave a smile on your face for days . xxbos xxmaj this is one of the movies selected to the xxup fifty xxup worst xxup films book by xxmaj harry xxmaj medved . xxmaj and while i agree it is really bad , xxmaj i 'm not sure it reached the level of wretchedness needed to merit inclusion in the book --"


In [8]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.3,metrics=[accuracy, Perplexity()])

In [9]:
#learn.lr_find()

In [10]:
#learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(3, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


### É bom salvar os modelos

In [ ]:
learn.save('1epoch')

### Para carregar os modelos e continuar o treinamento

In [ ]:
learn = learn.load('1epoch')

#learn.unfreeze()
#learn.fit_one_cycle(5, 2e-3)

### Salva o modelo final

In [ ]:
learn.save_encoder('finetuned')

### Adivinha a próxima palavra de uma sentença

In [ ]:
sentenca = "I liked this movie because"
n_palavras = 40
n_sentencas = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

### Classificação de sentenças utilizando LSTM

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=48, seq_len=72)

In [ ]:
dls_clas.show_batch(max_n=3)

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy)

In [ ]:
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.predict("I really loved that movie, it was awesome!")